# FMEA for Autonomous Vehicle Systems

**Failure Mode and Effects Analysis per ISO 26262**

Author: Milin Patel  
Institution: Hochschule Kempten - University of Applied Sciences

---

## Learning Objectives

1. Understand FMEA methodology in automotive context
2. Apply DFMEA and PFMEA to perception systems
3. Calculate Risk Priority Numbers (RPN)
4. Define corrective actions based on analysis

## 1. FMEA Fundamentals

### Types of FMEA in ISO 26262

| Type | Focus | Application Phase |
|------|-------|-------------------|
| **DFMEA** | Design | System/hardware/software design |
| **PFMEA** | Process | Manufacturing and assembly |
| **FMEDA** | Diagnostic | Hardware fault detection capability |

### FMEA Process Flow

```
System Definition → Failure Mode Identification → Effect Analysis → 
Severity Rating → Cause Analysis → Occurrence Rating → 
Detection Analysis → Detection Rating → RPN Calculation → 
Action Prioritization → Implementation → Re-evaluation
```

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import List, Dict, Optional
from enum import IntEnum

print("FMEA Analysis Tools Loaded")

## 2. FMEA Rating Scales

### Severity Scale (ISO 26262 Aligned)

In [ ]:
SEVERITY_SCALE = {
    10: "Hazardous - Fatal injury, ASIL D safety goal violation",
    9: "Serious - Severe injury likely, ASIL C safety goal violation",
    8: "Major - Potential injury, ASIL B safety goal violation",
    7: "Significant - Moderate injury possible, ASIL A violation",
    6: "Moderate - Minor injury possible, QM safety impact",
    5: "Low - Occupant discomfort, vehicle damage",
    4: "Very Low - Minor performance degradation",
    3: "Minor - Detectable degradation, no safety impact",
    2: "Very Minor - Slight degradation, barely detectable",
    1: "None - No effect"
}

OCCURRENCE_SCALE = {
    10: "Very High - Failure almost inevitable (>100 per 1000)",
    9: "High - Repeated failures (50 per 1000)",
    8: "High - Repeated failures (20 per 1000)",
    7: "Moderately High - Frequent failures (10 per 1000)",
    6: "Moderate - Occasional failures (5 per 1000)",
    5: "Moderate - Occasional failures (2 per 1000)",
    4: "Moderately Low - Infrequent failures (1 per 1000)",
    3: "Low - Few failures (0.5 per 1000)",
    2: "Very Low - Rare failures (0.1 per 1000)",
    1: "Remote - Failure unlikely (<0.01 per 1000)"
}

DETECTION_SCALE = {
    10: "Absolute Uncertainty - No detection possible",
    9: "Very Remote - Very unlikely to detect",
    8: "Remote - Detection unlikely",
    7: "Very Low - Very low detection probability",
    6: "Low - Low detection probability",
    5: "Moderate - Moderate detection probability",
    4: "Moderately High - Good detection probability",
    3: "High - High detection probability",
    2: "Very High - Very high detection probability",
    1: "Almost Certain - Detection almost certain"
}

def display_scale(scale: dict, title: str):
    """Display rating scale as table."""
    df = pd.DataFrame([
        {"Rating": k, "Description": v} 
        for k, v in sorted(scale.items(), reverse=True)
    ])
    print(f"\n{title}")
    print("=" * 60)
    for _, row in df.iterrows():
        print(f"{row['Rating']:2d}: {row['Description']}")

display_scale(SEVERITY_SCALE, "Severity Scale (S)")

## 3. FMEA Data Structure

In [ ]:
@dataclass
class FailureMode:
    """Represents a single failure mode entry in FMEA."""
    id: str
    component: str
    function: str
    failure_mode: str
    failure_effect_local: str
    failure_effect_system: str
    severity: int  # 1-10
    failure_cause: str
    occurrence: int  # 1-10
    current_controls: str
    detection: int  # 1-10
    recommended_actions: str = ""
    responsibility: str = ""
    target_completion: str = ""
    actions_taken: str = ""
    new_severity: Optional[int] = None
    new_occurrence: Optional[int] = None
    new_detection: Optional[int] = None
    
    @property
    def rpn(self) -> int:
        """Calculate Risk Priority Number."""
        return self.severity * self.occurrence * self.detection
    
    @property
    def new_rpn(self) -> Optional[int]:
        """Calculate new RPN after actions."""
        if all([self.new_severity, self.new_occurrence, self.new_detection]):
            return self.new_severity * self.new_occurrence * self.new_detection
        return None
    
    @property
    def criticality(self) -> int:
        """Criticality = Severity x Occurrence (per ISO 26262)."""
        return self.severity * self.occurrence

In [ ]:
@dataclass
class FMEAWorksheet:
    """FMEA Worksheet for systematic analysis."""
    system_name: str
    subsystem: str
    design_responsibility: str
    prepared_by: str
    fmea_date: str
    revision: str
    failure_modes: List[FailureMode] = field(default_factory=list)
    
    def add_failure_mode(self, fm: FailureMode):
        """Add failure mode to worksheet."""
        self.failure_modes.append(fm)
    
    def get_high_risk_items(self, rpn_threshold: int = 100) -> List[FailureMode]:
        """Get failure modes above RPN threshold."""
        return [fm for fm in self.failure_modes if fm.rpn >= rpn_threshold]
    
    def get_high_severity_items(self, severity_threshold: int = 8) -> List[FailureMode]:
        """Get failure modes with high severity regardless of RPN."""
        return [fm for fm in self.failure_modes if fm.severity >= severity_threshold]
    
    def to_dataframe(self) -> pd.DataFrame:
        """Convert to pandas DataFrame for analysis."""
        data = []
        for fm in self.failure_modes:
            data.append({
                'ID': fm.id,
                'Component': fm.component,
                'Function': fm.function,
                'Failure Mode': fm.failure_mode,
                'Local Effect': fm.failure_effect_local,
                'System Effect': fm.failure_effect_system,
                'S': fm.severity,
                'Cause': fm.failure_cause,
                'O': fm.occurrence,
                'Controls': fm.current_controls,
                'D': fm.detection,
                'RPN': fm.rpn,
                'Criticality': fm.criticality,
                'Recommended Actions': fm.recommended_actions
            })
        return pd.DataFrame(data)
    
    def summary_statistics(self) -> Dict:
        """Calculate summary statistics."""
        rpns = [fm.rpn for fm in self.failure_modes]
        return {
            'total_failure_modes': len(self.failure_modes),
            'max_rpn': max(rpns) if rpns else 0,
            'avg_rpn': np.mean(rpns) if rpns else 0,
            'high_risk_count': len(self.get_high_risk_items()),
            'high_severity_count': len(self.get_high_severity_items())
        }

## 4. DFMEA Example: Camera Perception System

In [ ]:
# Create FMEA worksheet for camera perception
camera_fmea = FMEAWorksheet(
    system_name="Autonomous Vehicle Perception System",
    subsystem="Forward-Facing Camera Module",
    design_responsibility="Perception Team",
    prepared_by="Safety Engineering",
    fmea_date="2025-01",
    revision="1.0"
)

# Define failure modes for camera system
camera_failure_modes = [
    FailureMode(
        id="CAM-001",
        component="Image Sensor",
        function="Capture visual scene for object detection",
        failure_mode="Complete image loss",
        failure_effect_local="No image data output",
        failure_effect_system="Loss of visual perception, potential collision",
        severity=9,
        failure_cause="Sensor hardware failure, power loss",
        occurrence=2,
        current_controls="Hardware redundancy, power monitoring",
        detection=2,
        recommended_actions="Implement dual-camera redundancy with automatic failover"
    ),
    FailureMode(
        id="CAM-002",
        component="Image Sensor",
        function="Capture visual scene for object detection",
        failure_mode="Degraded image quality (noise, blur)",
        failure_effect_local="Reduced image clarity",
        failure_effect_system="Degraded detection performance, false negatives",
        severity=7,
        failure_cause="Sensor aging, temperature effects, vibration",
        occurrence=4,
        current_controls="Image quality monitoring algorithm",
        detection=4,
        recommended_actions="Add predictive maintenance alerts, enhance quality metrics"
    ),
    FailureMode(
        id="CAM-003",
        component="Lens Assembly",
        function="Focus light onto sensor",
        failure_mode="Lens obstruction (dirt, ice, debris)",
        failure_effect_local="Partial or full image blockage",
        failure_effect_system="Blind spots in perception, missed objects",
        severity=8,
        failure_cause="Environmental contamination, inadequate cleaning",
        occurrence=5,
        current_controls="Obstruction detection algorithm",
        detection=3,
        recommended_actions="Implement automatic lens cleaning system"
    ),
    FailureMode(
        id="CAM-004",
        component="Neural Network",
        function="Detect and classify objects",
        failure_mode="Misclassification of objects",
        failure_effect_local="Incorrect object labels",
        failure_effect_system="Wrong decision-making, inappropriate response",
        severity=8,
        failure_cause="Training data bias, OOD inputs, adversarial patterns",
        occurrence=4,
        current_controls="Confidence thresholding",
        detection=5,
        recommended_actions="Implement uncertainty estimation, OOD detection"
    ),
    FailureMode(
        id="CAM-005",
        component="Neural Network",
        function="Detect and classify objects",
        failure_mode="False negative (missed detection)",
        failure_effect_local="Object not reported",
        failure_effect_system="Collision with undetected object",
        severity=10,
        failure_cause="Low contrast, occlusion, unusual appearance",
        occurrence=3,
        current_controls="Multi-sensor fusion",
        detection=4,
        recommended_actions="Enhance training data diversity, add LiDAR backup"
    ),
    FailureMode(
        id="CAM-006",
        component="Neural Network",
        function="Detect and classify objects",
        failure_mode="False positive (phantom detection)",
        failure_effect_local="Non-existent object reported",
        failure_effect_system="Unnecessary braking, rear-end collision risk",
        severity=7,
        failure_cause="Visual artifacts, reflections, shadows",
        occurrence=4,
        current_controls="Temporal consistency check",
        detection=3,
        recommended_actions="Multi-frame validation, radar cross-check"
    ),
    FailureMode(
        id="CAM-007",
        component="Processing Unit",
        function="Execute perception algorithms",
        failure_mode="Processing latency exceeded",
        failure_effect_local="Delayed object detection",
        failure_effect_system="Stale perception data, delayed response",
        severity=7,
        failure_cause="Computational overload, thermal throttling",
        occurrence=3,
        current_controls="Watchdog timer, performance monitoring",
        detection=2,
        recommended_actions="Implement graceful degradation mode"
    ),
    FailureMode(
        id="CAM-008",
        component="Calibration Data",
        function="Provide accurate spatial mapping",
        failure_mode="Calibration drift/loss",
        failure_effect_local="Incorrect spatial measurements",
        failure_effect_system="Distance estimation errors, path planning failures",
        severity=8,
        failure_cause="Mechanical shock, temperature cycling",
        occurrence=3,
        current_controls="Periodic calibration verification",
        detection=4,
        recommended_actions="Online self-calibration, redundant reference points"
    )
]

for fm in camera_failure_modes:
    camera_fmea.add_failure_mode(fm)

print(f"Created FMEA with {len(camera_fmea.failure_modes)} failure modes")

In [ ]:
# Display FMEA worksheet
df = camera_fmea.to_dataframe()

# Display key columns
display_cols = ['ID', 'Component', 'Failure Mode', 'S', 'O', 'D', 'RPN', 'Criticality']
df_display = df[display_cols].sort_values('RPN', ascending=False)

print("Camera Perception FMEA Summary")
print("=" * 80)
print(df_display.to_string(index=False))

## 5. RPN Analysis and Visualization

In [ ]:
def plot_rpn_analysis(fmea: FMEAWorksheet):
    """Visualize FMEA risk analysis."""
    df = fmea.to_dataframe()
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. RPN Pareto Chart
    ax1 = axes[0, 0]
    df_sorted = df.sort_values('RPN', ascending=False)
    colors = ['#d32f2f' if rpn >= 100 else '#ff9800' if rpn >= 50 else '#4caf50' 
              for rpn in df_sorted['RPN']]
    bars = ax1.bar(range(len(df_sorted)), df_sorted['RPN'], color=colors)
    ax1.set_xticks(range(len(df_sorted)))
    ax1.set_xticklabels(df_sorted['ID'], rotation=45, ha='right')
    ax1.axhline(y=100, color='r', linestyle='--', label='High Risk Threshold')
    ax1.axhline(y=50, color='orange', linestyle='--', label='Medium Risk Threshold')
    ax1.set_ylabel('Risk Priority Number (RPN)')
    ax1.set_title('RPN Ranking')
    ax1.legend()
    
    # 2. Severity vs Occurrence Matrix
    ax2 = axes[0, 1]
    scatter = ax2.scatter(df['O'], df['S'], s=df['RPN']*2, 
                          c=df['D'], cmap='RdYlGn_r', alpha=0.7)
    for i, row in df.iterrows():
        ax2.annotate(row['ID'], (row['O'], row['S']), fontsize=8)
    ax2.set_xlabel('Occurrence (O)')
    ax2.set_ylabel('Severity (S)')
    ax2.set_title('Severity vs Occurrence (size=RPN, color=Detection)')
    ax2.set_xlim(0, 11)
    ax2.set_ylim(0, 11)
    ax2.axhline(y=8, color='r', linestyle=':', alpha=0.5)
    ax2.axvline(x=5, color='orange', linestyle=':', alpha=0.5)
    plt.colorbar(scatter, ax=ax2, label='Detection Rating')
    
    # 3. S-O-D Radar for top risks
    ax3 = axes[1, 0]
    top_risks = df.nlargest(4, 'RPN')
    categories = ['Severity', 'Occurrence', 'Detection']
    angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1]
    
    for _, row in top_risks.iterrows():
        values = [row['S'], row['O'], row['D']]
        values += values[:1]
        ax3.plot(angles, values, 'o-', label=row['ID'])
        ax3.fill(angles, values, alpha=0.1)
    
    ax3.set_xticks(angles[:-1])
    ax3.set_xticklabels(categories)
    ax3.set_ylim(0, 10)
    ax3.set_title('S-O-D Profile (Top 4 Risks)')
    ax3.legend(loc='upper right', fontsize=8)
    
    # 4. Component Risk Distribution
    ax4 = axes[1, 1]
    component_risk = df.groupby('Component')['RPN'].sum().sort_values(ascending=True)
    component_risk.plot(kind='barh', ax=ax4, color='steelblue')
    ax4.set_xlabel('Total RPN')
    ax4.set_title('Risk by Component')
    
    plt.tight_layout()
    plt.show()

plot_rpn_analysis(camera_fmea)

## 6. FMEA to ASIL Mapping

Linking FMEA severity to ASIL determination from HARA.

In [ ]:
def map_severity_to_hara(severity: int) -> str:
    """Map FMEA severity to HARA severity class."""
    if severity >= 9:
        return "S3 - Life-threatening/Fatal"
    elif severity >= 7:
        return "S2 - Severe/Life-threatening"
    elif severity >= 5:
        return "S1 - Light/Moderate"
    else:
        return "S0 - No injuries"

def suggest_safety_mechanism(failure_mode: FailureMode) -> List[str]:
    """Suggest safety mechanisms based on failure characteristics."""
    suggestions = []
    
    # High severity needs robust mechanisms
    if failure_mode.severity >= 9:
        suggestions.append("Hardware redundancy with independent channels")
        suggestions.append("Fail-safe state transition")
    
    # High occurrence needs preventive measures
    if failure_mode.occurrence >= 5:
        suggestions.append("Design improvements to reduce failure rate")
        suggestions.append("Enhanced environmental protection")
    
    # Low detection needs monitoring
    if failure_mode.detection >= 6:
        suggestions.append("Online monitoring and diagnostics")
        suggestions.append("Cross-validation with redundant sensors")
    
    # Neural network specific
    if "Neural Network" in failure_mode.component:
        suggestions.append("Uncertainty estimation (MC Dropout, Ensembles)")
        suggestions.append("Runtime OOD detection")
        suggestions.append("Plausibility checks with other sensors")
    
    return suggestions if suggestions else ["Standard quality controls sufficient"]

# Analyze top risks
print("Safety Mechanism Recommendations")
print("=" * 70)

for fm in camera_fmea.get_high_risk_items(rpn_threshold=80):
    print(f"\n{fm.id}: {fm.failure_mode}")
    print(f"  HARA Severity: {map_severity_to_hara(fm.severity)}")
    print(f"  RPN: {fm.rpn}")
    print(f"  Recommended Safety Mechanisms:")
    for mechanism in suggest_safety_mechanism(fm):
        print(f"    - {mechanism}")

## 7. FMEDA for Hardware Metrics

Failure Modes, Effects, and Diagnostic Analysis for calculating diagnostic coverage.

In [ ]:
@dataclass
class FMEDAEntry:
    """FMEDA entry for hardware diagnostic coverage calculation."""
    component: str
    failure_mode: str
    failure_rate_fit: float  # FIT = Failures in Time (per 10^9 hours)
    failure_mode_distribution: float  # Percentage of total failures
    safe_fault: bool
    detected: bool
    diagnostic_coverage: float  # 0-1
    
    @property
    def effective_failure_rate(self) -> float:
        """Calculate effective failure rate considering distribution."""
        return self.failure_rate_fit * self.failure_mode_distribution

def calculate_hardware_metrics(entries: List[FMEDAEntry]) -> Dict:
    """Calculate ISO 26262 hardware metrics from FMEDA."""
    total_failure_rate = sum(e.effective_failure_rate for e in entries)
    
    safe_faults = sum(e.effective_failure_rate for e in entries if e.safe_fault)
    detected_faults = sum(e.effective_failure_rate * e.diagnostic_coverage 
                         for e in entries if not e.safe_fault)
    residual_faults = sum(e.effective_failure_rate * (1 - e.diagnostic_coverage) 
                         for e in entries if not e.safe_fault and not e.safe_fault)
    
    # Single Point Fault Metric (SPFM)
    dangerous_faults = total_failure_rate - safe_faults
    spfm = 1 - (residual_faults / dangerous_faults) if dangerous_faults > 0 else 1
    
    # Latent Fault Metric (LFM) - simplified
    lfm = detected_faults / dangerous_faults if dangerous_faults > 0 else 1
    
    return {
        'total_failure_rate_fit': total_failure_rate,
        'safe_fault_fraction': safe_faults / total_failure_rate if total_failure_rate > 0 else 0,
        'spfm': spfm,
        'lfm': lfm,
        'asil_b_spfm_target': 0.90,
        'asil_c_spfm_target': 0.97,
        'asil_d_spfm_target': 0.99
    }

# Example FMEDA for camera sensor
camera_fmeda = [
    FMEDAEntry("Image Sensor", "No output", 50, 0.3, False, True, 0.99),
    FMEDAEntry("Image Sensor", "Stuck pixels", 50, 0.2, True, False, 0.0),  # Safe fault
    FMEDAEntry("Image Sensor", "Noise increase", 50, 0.3, False, True, 0.85),
    FMEDAEntry("Image Sensor", "Timing drift", 50, 0.2, False, True, 0.95),
    FMEDAEntry("Power Supply", "Overvoltage", 20, 0.4, False, True, 0.99),
    FMEDAEntry("Power Supply", "Undervoltage", 20, 0.4, False, True, 0.99),
    FMEDAEntry("Power Supply", "Ripple", 20, 0.2, False, True, 0.80),
    FMEDAEntry("Communication", "Bit flip", 10, 0.5, False, True, 0.999),  # CRC
    FMEDAEntry("Communication", "Message loss", 10, 0.5, False, True, 0.95),
]

metrics = calculate_hardware_metrics(camera_fmeda)

print("FMEDA Hardware Metrics")
print("=" * 50)
print(f"Total Failure Rate: {metrics['total_failure_rate_fit']:.1f} FIT")
print(f"Safe Fault Fraction: {metrics['safe_fault_fraction']:.1%}")
print(f"\nSingle Point Fault Metric (SPFM): {metrics['spfm']:.2%}")
print(f"  ASIL B target (>=90%): {'PASS' if metrics['spfm'] >= 0.90 else 'FAIL'}")
print(f"  ASIL C target (>=97%): {'PASS' if metrics['spfm'] >= 0.97 else 'FAIL'}")
print(f"  ASIL D target (>=99%): {'PASS' if metrics['spfm'] >= 0.99 else 'FAIL'}")
print(f"\nLatent Fault Metric (LFM): {metrics['lfm']:.2%}")

## 8. Action Tracking and Risk Reduction

In [ ]:
def simulate_action_effectiveness(fm: FailureMode, 
                                  severity_reduction: int = 0,
                                  occurrence_reduction: int = 0,
                                  detection_improvement: int = 0) -> Dict:
    """Simulate effect of corrective actions on RPN."""
    new_s = max(1, fm.severity - severity_reduction)
    new_o = max(1, fm.occurrence - occurrence_reduction)
    new_d = max(1, fm.detection - detection_improvement)
    
    original_rpn = fm.rpn
    new_rpn = new_s * new_o * new_d
    
    return {
        'original_rpn': original_rpn,
        'new_rpn': new_rpn,
        'reduction': original_rpn - new_rpn,
        'reduction_pct': (original_rpn - new_rpn) / original_rpn * 100,
        'new_s': new_s,
        'new_o': new_o,
        'new_d': new_d
    }

# Example: Evaluate action effectiveness for top risk
top_risk = camera_fmea.get_high_risk_items()[0]

print(f"Action Effectiveness Analysis: {top_risk.id}")
print(f"Failure Mode: {top_risk.failure_mode}")
print(f"Original: S={top_risk.severity}, O={top_risk.occurrence}, D={top_risk.detection}, RPN={top_risk.rpn}")
print()

# Scenario 1: Improve detection
result1 = simulate_action_effectiveness(top_risk, detection_improvement=2)
print(f"Scenario 1 (Detection +2): RPN {result1['original_rpn']} -> {result1['new_rpn']} ({result1['reduction_pct']:.0f}% reduction)")

# Scenario 2: Reduce occurrence
result2 = simulate_action_effectiveness(top_risk, occurrence_reduction=2)
print(f"Scenario 2 (Occurrence -2): RPN {result2['original_rpn']} -> {result2['new_rpn']} ({result2['reduction_pct']:.0f}% reduction)")

# Scenario 3: Combined approach
result3 = simulate_action_effectiveness(top_risk, occurrence_reduction=1, detection_improvement=1)
print(f"Scenario 3 (Combined):     RPN {result3['original_rpn']} -> {result3['new_rpn']} ({result3['reduction_pct']:.0f}% reduction)")

## 9. FMEA Report Generation

In [ ]:
def generate_fmea_report(fmea: FMEAWorksheet) -> str:
    """Generate FMEA summary report."""
    stats = fmea.summary_statistics()
    df = fmea.to_dataframe()
    
    report = f"""
FMEA SUMMARY REPORT
{'=' * 60}

System: {fmea.system_name}
Subsystem: {fmea.subsystem}
Prepared by: {fmea.prepared_by}
Date: {fmea.fmea_date}
Revision: {fmea.revision}

SUMMARY STATISTICS
{'-' * 40}
Total Failure Modes Analyzed: {stats['total_failure_modes']}
Maximum RPN: {stats['max_rpn']}
Average RPN: {stats['avg_rpn']:.1f}
High Risk Items (RPN >= 100): {stats['high_risk_count']}
High Severity Items (S >= 8): {stats['high_severity_count']}

TOP PRIORITY ITEMS (Action Required)
{'-' * 40}
"""
    
    for fm in fmea.get_high_risk_items(rpn_threshold=80):
        report += f"""
ID: {fm.id}
  Component: {fm.component}
  Failure Mode: {fm.failure_mode}
  Effect: {fm.failure_effect_system}
  S/O/D: {fm.severity}/{fm.occurrence}/{fm.detection}
  RPN: {fm.rpn}
  Recommended Action: {fm.recommended_actions}
"""
    
    report += f"""
RISK DISTRIBUTION BY COMPONENT
{'-' * 40}
"""
    component_risk = df.groupby('Component')['RPN'].agg(['sum', 'count', 'mean'])
    for comp, row in component_risk.iterrows():
        report += f"{comp}: {row['count']:.0f} modes, Total RPN: {row['sum']:.0f}, Avg: {row['mean']:.1f}\n"
    
    return report

print(generate_fmea_report(camera_fmea))

## 10. Key Takeaways

### FMEA Best Practices

1. **Start Early**: Begin FMEA during concept phase, update throughout development
2. **Cross-Functional Team**: Include design, safety, test, and manufacturing expertise
3. **Be Specific**: Generic failure modes yield poor analysis
4. **Prioritize Actions**: Focus on high-severity items first, then high RPN
5. **Track Effectiveness**: Re-evaluate RPN after implementing actions

### ISO 26262 Integration

- FMEA severity links to HARA severity classification
- FMEDA provides hardware metrics (SPFM, LFM) for ASIL decomposition
- Safety mechanisms identified in FMEA feed into safety concept
- Failure modes inform test case development

### Perception System Considerations

- Neural network failures (misclassification, missed detection) require special treatment
- Environmental factors (weather, lighting) increase occurrence ratings
- Sensor fusion provides detection improvement across components
- Runtime monitoring essential for ML-based components

## References

1. ISO 26262:2018 - Road vehicles - Functional safety
2. SAE J1739 - Potential Failure Mode and Effects Analysis
3. AIAG & VDA FMEA Handbook (2019)
4. IEC 60812 - Analysis techniques for system reliability